# Tricking Stats

In [8]:
import pandas as pd
tricking = pd.read_csv('Tricking.csv')
tricking

,Date,Trick_Name,Attempts,Landed_Count,Surface
0,2025-09-25,Fulls,3,3,Spring Floor
1,2025-09-25,Full Hyperhook,7,7,Spring Floor
2,2025-09-25,Double Full,7,6,Spring Floor
3,2025-09-25,Corks,3,3,Spring Floor
4,2025-09-25,Cheat 1080,2,2,Spring Floor
...,...,...,...,...,...
160,2025-12-07,Full Hyperhook,2,2,Spring Floor
161,2025-12-07,Double Full,2,1,Spring Floor
162,2025-12-07,Double Full Hyperhook,5,0,Spring Floor
163,2025-12-07,Corks,4,4,Spring Floor


In [9]:
tricking['Trick_Name'].unique()

array(['Fulls', 'Full Hyperhook', 'Double Full', 'Corks', 'Cheat 1080',
       'Double B-Twist', 'Cheat 900', 'Gainer', 'Corks (Round)',
       'Round Off Backflip (Multiple Kicks)', 'Double Full Hyperhook',
       'Full Snapu', 'B-Twist Hyperhook', 'B-Twist', 'Double Cork',
       'Round Off Backflip', 'Boxcutter', 'Full Hyperhook (Shuriken)',
       'Triple Full', 'B-Twist (Shuriken)', 'Front Tuck',
       'Back Handspring', 'Wrap Full', 'Wrap Full Hyperhook',
       'Backside 1260', 'Round Off Backflip ', 'Side Swipe',
       'Gainer (Switch)'], dtype=object)

### Tricks I've done most often

In [10]:
tricking['Trick_Name'].value_counts().head() # Value

Trick_Name
Full Hyperhook    18
Corks             17
Fulls             14
Cheat 1080        13
Double Full       12
Name: count, dtype: int64

In [11]:
tricking['Trick_Name'].value_counts(normalize = True).head() # Percentage

Trick_Name
Full Hyperhook    0.109091
Corks             0.103030
Fulls             0.084848
Cheat 1080        0.078788
Double Full       0.072727
Name: proportion, dtype: float64

### Show tricks that I didn't land 100% of the time

In [12]:
nlanded100 = tricking['Attempts'] != tricking['Landed_Count']
tricking[nlanded100].head()

,Date,Trick_Name,Attempts,Landed_Count,Surface
2,2025-09-25,Double Full,7,6,Spring Floor
7,2025-09-28,Double B-Twist,3,0,Spring Floor
11,2025-10-05,Corks (Round),7,5,Spring Floor
12,2025-10-05,Cheat 1080,5,3,Spring Floor
22,2025-10-08,Round Off Backflip (Multiple Kicks),3,2,Puzzle Mats


In [13]:
tricking['Attempts'].describe()

count    165.000000
mean       2.339394
std        1.945906
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       16.000000
Name: Attempts, dtype: float64

In [14]:
tricking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          165 non-null    object
 1   Trick_Name    165 non-null    object
 2   Attempts      165 non-null    int64 
 3   Landed_Count  165 non-null    int64 
 4   Surface       165 non-null    object
dtypes: int64(2), object(3)
memory usage: 6.6+ KB
